![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Training and Reusing Assertion Status Models

In [0]:
import os
import json
import string
import numpy as np
import pandas as pd


import sparknlp
import sparknlp_jsl
from sparknlp.base import *
from sparknlp.util import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.pretrained import ResourceDownloader

from pyspark.sql import functions as F
from pyspark.ml import Pipeline, PipelineModel

pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)  
pd.set_option('display.expand_frame_repr', False)

print('sparknlp.version : ',sparknlp.version())
print('sparknlp_jsl.version : ',sparknlp_jsl.version())

spark

sparknlp.version : 3.4.2
sparknlp_jsl.version : 3.5.1
Out[1]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.1.2 
 Master 
 spark://10.139.64.6:7077 
 AppName 
 Databricks Shell

# Clinical Assertion Model (with pretrained models)

The deep neural network architecture for assertion status detection in Spark NLP is based on a Bi-LSTM framework, and is a modified version of the architecture proposed by Federico Fancellu, Adam Lopez and Bonnie Webber ([Neural Networks For Negation Scope Detection](https://aclanthology.org/P16-1047.pdf)). Its goal is to classify the assertions made on given medical concepts as being present, absent, or possible in the patient, conditionally present in the patient under certain circumstances,
hypothetically present in the patient at some future point, and
mentioned in the patient report but associated with someoneelse.
In the proposed implementation, input units depend on the
target tokens (a named entity) and the neighboring words that
are explicitly encoded as a sequence using word embeddings.
Similar to paper mentioned above,  it is observed that that 95% of the scope tokens (neighboring words) fall in a window of 9 tokens to the left and 15
to the right of the target tokens in the same dataset. Therefore, the same window size was implemented and it following parameters were used: learning
rate 0.0012, dropout 0.05, batch size 64 and a maximum sentence length 250. The model has been implemented within
Spark NLP as an annotator called AssertionDLModel. After
training 20 epoch and measuring accuracy on the official test
set, this implementation exceeds the latest state-of-the-art
accuracy benchmarks as summarized as following table:

|Assertion Label|Spark NLP|Latest Best|
|-|-|-|
|Absent       |0.944 |0.937|
|Someone-else |0.904|0.869|
|Conditional  |0.441|0.422|
|Hypothetical |0.862|0.890|
|Possible     |0.680|0.630|
|Present      |0.953|0.957|
|micro F1     |0.939|0.934|

|    | model_name              |Predicted Entities|
|---:|:------------------------|-|
|  1 | assertion_dl            |Present, Absent, Possible, Planned, Someoneelse, Past, Family, None, Hypotetical|
|  2 | assertion_dl_biobert    |absent, present, conditional, associated_with_someone_else, hypothetical, possible|
|  3 | assertion_dl_healthcare |absent, present, conditional, associated_with_someone_else, hypothetical, possible|
|  4 | assertion_dl_large      |hypothetical, present, absent, possible, conditional, associated_with_someone_else|
|  5 | assertion_dl_radiology  |Confirmed, Suspected, Negative|
|  6 | assertion_jsl           |Present, Absent, Possible, Planned, Someoneelse, Past, Family, None, Hypotetical|
|  7 | assertion_jsl_large     |present, absent, possible, planned, someoneelse, past|
|  8 | assertion_ml            |Hypothetical, Present, Absent, Possible, Conditional, Associated_with_someone_else|

## Pretrained `assertion_dl` model

In [0]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP

from sparknlp_jsl.annotator import *

documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line

sentenceDetector = SentenceDetector()\
  .setInputCols(["document"])\
  .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP

tokenizer = Tokenizer()\
  .setInputCols(["sentence"])\
  .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
  .setInputCols(["sentence", "token"])\
  .setOutputCol("embeddings")

# NER model trained on i2b2 (sampled from MIMIC) dataset
clinical_ner = MedicalNerModel.pretrained("ner_clinical", "en", "clinical/models") \
  .setInputCols(["sentence", "token", "embeddings"]) \
  .setOutputCol("ner")

ner_converter = NerConverter() \
  .setInputCols(["sentence", "token", "ner"]) \
  .setOutputCol("ner_chunk")

# Assertion model trained on i2b2 (sampled from MIMIC) dataset
# coming from sparknlp_jsl.annotator !!
clinical_assertion = AssertionDLModel.pretrained("assertion_dl", "en", "clinical/models") \
  .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
  .setOutputCol("assertion")

nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    clinical_ner,
    ner_converter,
    clinical_assertion
    ])

empty_data = spark.createDataFrame([[""]]).toDF("text")
model = nlpPipeline.fit(empty_data)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_clinical download started this may take some time.
Approximate size to download 13.9 MB
[ | ][OK!]
assertion_dl download started this may take some time.
Approximate size to download 1.3 MB
[ | ][OK!]

In [0]:
# we also have a LogReg based Assertion Model.
'''
clinical_assertion_ml = AssertionLogRegModel.pretrained("assertion_ml", "en", "clinical/models") \
    .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")
'''

Out[3]: '\nclinical_assertion_ml = AssertionLogRegModel.pretrained("assertion_ml", "en", "clinical/models") .setInputCols(["sentence", "ner_chunk", "embeddings"]) .setOutputCol("assertion")\n'

In [0]:
import pandas as pd

text = 'Patient has a headache for the last 2 weeks, needs to get a head CT, and appears anxious when she walks fast. No alopecia noted. She denies pain'

print (text)

light_model = LightPipeline(model)

light_result = light_model.fullAnnotate(text)[0]

chunks=[]
entities=[]
status=[]

for n,m in zip(light_result['ner_chunk'],light_result['assertion']):
    
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    status.append(m.result)
        
df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status})

df

Patient has a headache for the last 2 weeks, needs to get a head CT, and appears anxious when she walks fast. No alopecia noted. She denies pain
Out[4]:

,chunks,entities,assertion
0,a headache,PROBLEM,present
1,a head CT,TEST,present
2,anxious,PROBLEM,present
3,alopecia,PROBLEM,absent
4,pain,PROBLEM,absent


In [0]:
from sparknlp_display import AssertionVisualizer

visualizer = AssertionVisualizer()

vis = visualizer.display(light_result, 'ner_chunk', 'assertion', return_html=True)
#visualizer.set_label_colors({'TREATMENT':'#008080', 'PROBLEM':'#800080'})


displayHTML(vis)

Patient has a headache PROBLEM present for the last 2 weeks, needs to get a head CT TEST present , and appears anxious PROBLEM present when she walks fast. No alopecia PROBLEM absent noted. She denies pain PROBLEM absent

In [0]:
! wget -q	https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/pubmed/pubmed_sample_text_small.csv -P /dbfs/

In [0]:
import pyspark.sql.functions as F

pubMedDF = spark.read\
                .option("header", "true")\
                .csv("/pubmed_sample_text_small.csv")\
                
pubMedDF.show(truncate=50)

+--------------------------------------------------+
 text|
+--------------------------------------------------+
The human KCNJ9 (Kir 3.3, GIRK3) is a member of...|
BACKGROUND: At present, it is one of the most i...|
OBJECTIVE: To investigate the relationship betw...|
Combined EEG/fMRI recording has been used to lo...|
Kohlschutter syndrome is a rare neurodegenerati...|
Statistical analysis of neuroimages is commonly...|
The synthetic DOX-LNA conjugate was characteriz...|
Our objective was to compare three different me...|
We conducted a phase II study to assess the eff...|
"""Monomeric sarcosine oxidase (MSOX) is a flav...|
We presented the tachinid fly Exorista japonica...|
The literature dealing with the water conductin...|
A novel approach to synthesize chitosan-O-isopr...|
An HPLC-ESI-MS-MS method has been developed for...|
The localizing and lateralizing values of eye a...|
OBJECTIVE: To evaluate the effectiveness and ac...|
For the construction of new combinatorial libra...|
We report the results of a screen for genetic a...|
Intraparenchymal pericatheter cyst is rarely re...|
It is known that patients with Klinefelter's sy...|
+--------------------------------------------------+
only showing top 20 rows

In [0]:
result = model.transform(pubMedDF.limit(100))

In [0]:
result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| embeddings| ner| ner_chunk| assertion|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
The human KCNJ9 (...|[{document, 0, 95...|[{document, 0, 12...|[{token, 0, 2, Th...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 48, 106,...|[{assertion, 48, ...|
BACKGROUND: At pr...|[{document, 0, 14...|[{document, 0, 19...|[{token, 0, 9, BA...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 67, 79, ...|[{assertion, 67, ...|
OBJECTIVE: To inv...|[{document, 0, 15...|[{document, 0, 30...|[{token, 0, 8, OB...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 188, 231...|[{assertion, 188,...|
Combined EEG/fMRI...|[{document, 0, 16...|[{document, 0, 16...|[{token, 0, 7, Co...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 26, C...|[{assertion, 0, 2...|
Kohlschutter synd...|[{document, 0, 25...|[{document, 0, 20...|[{token, 0, 11, K...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 20, K...|[{assertion, 0, 2...|
Statistical analy...|[{document, 0, 10...|[{document, 0, 34...|[{token, 0, 10, S...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 34, S...|[{assertion, 0, 3...|
The synthetic DOX...|[{document, 0, 57...|[{document, 0, 10...|[{token, 0, 2, Th...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 30, T...|[{assertion, 0, 3...|
Our objective was...|[{document, 0, 24...|[{document, 0, 19...|[{token, 0, 2, Ou...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 56, 80, ...|[{assertion, 56, ...|
We conducted a ph...|[{document, 0, 14...|[{document, 0, 24...|[{token, 0, 1, We...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 13, 28, ...|[{assertion, 13, ...|
"""Monomeric sarc...|[{document, 0, 14...|[{document, 0, 14...|[{token, 0, 2, ""...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 3, 36, M...|[{assertion, 3, 3...|
We presented the ...|[{document, 0, 12...|[{document, 0, 26...|[{token, 0, 1, We...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 13, 46, ...|[{assertion, 13, ...|
The literature de...|[{document, 0, 16...|[{document, 0, 13...|[{token, 0, 2, Th...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 154, 166...|[{assertion, 154,...|
A novel approach ...|[{document, 0, 64...|[{document, 0, 97...|[{token, 0, 0, A,...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 99, 124,...|[{assertion, 99, ...|
An HPLC-ESI-MS-MS...|[{document, 0, 90...|[{document, 0, 24...|[{token, 0, 1, An...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 23, A...|[{assertion, 0, 2...|
The localizing an...|[{document, 0, 72...|[{document, 0, 12...|[{token, 0, 2, Th...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 50, 70, ...|[{assertion, 50, ...|
OBJECTIVE: To eva...|[{document, 0, 13...|[{document, 0, 43...|[{token, 0, 8, OB...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 154, 186...|[{assertion, 154,...|
For the construct...|[{document, 0, 32...|[{document, 0, 20...|[{token, 0, 2, Fo...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 24, 50, ...|[{assertion, 24, ...|
We report the res...|[{document, 0, 13...|[{document, 0, 38...|[{token, 0, 1, We...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 25, 32, ...|[{assertion, 25, ...|
Intraparenchymal ...|[{document, 0, 10...|[{document, 0, 53...|[{token, 0, 15, I...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 33, I...|[{assertion, 0, 3...|
It is known that ...|[{document, 0, 34...|[{document, 0, 14...|[{token, 0, 1, It...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 31, 52, ...|[{assertion, 31, ...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [0]:
result.select('sentence.result').take(1)

Out[10]: [Row(result=['The human KCNJ9 (Kir 3.3, GIRK3) is a member of the G-protein-activated inwardly rectifying potassium (GIRK) channel family.', 'Here we describe the genomicorganization of the KCNJ9 locus on chromosome 1q21-23 as a candidate gene forType II diabetes mellitus in the Pima Indian population.', 'The gene spansapproximately 7.6 kb and contains one noncoding and two coding exons separated byapproximately 2.2 and approximately 2.6 kb introns, respectively.', 'We identified14 single nucleotide polymorphisms (SNPs), including one that predicts aVal366Ala substitution, and an 8 base-pair', '(bp) insertion/deletion.', 'Ourexpression studies revealed the presence of the transcript in various humantissues including pancreas, and two major insulin-responsive tissues: fat andskeletal muscle.', 'The characterization of the KCNJ9 gene should facilitate furtherstudies on the function of the KCNJ9 protein and allow evaluation of thepotential role of the locus in Type II diabetes.'])]

In [0]:
result.select(F.explode(F.arrays_zip(result.ner_chunk.result,
                                     result.ner_chunk.begin, 
                                     result.ner_chunk.end,
                                     result.ner_chunk.metadata,
                                     result.assertion.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']").alias("begin"),
              F.expr("cols['2']").alias("end"),
              F.expr("cols['3']['entity']").alias("ner_label"),
              F.expr("cols['3']['sentence']").alias("sent_id"),
              F.expr("cols['4']").alias("assertion"),).show(truncate=False)


+-----------------------------------------------------------+-----+---+---------+-------+-----------+
chunk |begin|end|ner_label|sent_id|assertion |
+-----------------------------------------------------------+-----+---+---------+-------+-----------+
the G-protein-activated inwardly rectifying potassium (GIRK|48 |106|TREATMENT|0 |conditional|
the genomicorganization |142 |164|TREATMENT|1 |present |
a candidate gene forType II diabetes mellitus |210 |254|PROBLEM |1 |present |
byapproximately |380 |394|TREATMENT|2 |present |
single nucleotide polymorphisms |464 |494|TREATMENT|3 |present |
aVal366Ala substitution |532 |554|PROBLEM |3 |present |
an 8 base-pair |561 |574|PROBLEM |3 |present |
insertion/deletion |581 |598|PROBLEM |4 |present |
Ourexpression studies |601 |621|TEST |5 |present |
the transcript in various humantissues |648 |685|PROBLEM |5 |present |
fat andskeletal muscle |749 |770|PROBLEM |5 |possible |
furtherstudies |830 |843|PROBLEM |6 |present |
the KCNJ9 protein |864 |880|TREATMENT|6 |present |
evaluation |892 |901|TEST |6 |possible |
Type II diabetes |940 |955|PROBLEM |6 |present |
the treatment |67 |79 |TREATMENT|0 |present |
breast cancer |84 |96 |PROBLEM |0 |present |
the standard therapy |109 |128|TREATMENT|0 |present |
anthracyclines |167 |180|TREATMENT|0 |present |
taxanes |186 |192|TREATMENT|0 |present |
+-----------------------------------------------------------+-----+---+---------+-------+-----------+
only showing top 20 rows

## Pretrained `assertion_dl_radiology` model

In [0]:
from sparknlp_jsl.annotator import *

documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line
sentenceDetector = SentenceDetectorDLModel\
    .pretrained("sentence_detector_dl_healthcare","en","clinical/models") \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

# NER model for radiology
radiology_ner = MedicalNerModel.pretrained("ner_radiology", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")

ner_converter = NerConverter() \
    .setInputCols(["sentence", "token", "ner"]) \
    .setOutputCol("ner_chunk")\
    .setWhiteList(["ImagingFindings"])

# Assertion model trained on radiology dataset
# coming from sparknlp_jsl.annotator !!

radiology_assertion = AssertionDLModel.pretrained("assertion_dl_radiology", "en", "clinical/models") \
    .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")

nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    radiology_ner,
    ner_converter,
    radiology_assertion
    ])

empty_data = spark.createDataFrame([[""]]).toDF("text")
radiologyAssertion_model = nlpPipeline.fit(empty_data)

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_radiology download started this may take some time.
Approximate size to download 13.9 MB
[ | ][OK!]
assertion_dl_radiology download started this may take some time.
Approximate size to download 1.3 MB
[ | ][OK!]

In [0]:
# A sample text from a radiology report

text = """No right-sided pleural effusion or pneumothorax is definitively seen and there are mildly displaced fractures of the left lateral 8th and likely 9th ribs."""

In [0]:
data = spark.createDataFrame([[text]]).toDF("text")

In [0]:
result = radiologyAssertion_model.transform(data)

In [0]:
result.select(F.explode(F.arrays_zip(result.ner_chunk.result, 
                                     result.ner_chunk.metadata, 
                                     result.assertion.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label"),
              F.expr("cols['1']['sentence']").alias("sent_id"),
              F.expr("cols['2']").alias("assertion")).show(truncate=False)

+-------------------+---------------+-------+---------+
chunk |ner_label |sent_id|assertion|
+-------------------+---------------+-------+---------+
effusion |ImagingFindings|0 |Negative |
pneumothorax |ImagingFindings|0 |Negative |
displaced fractures|ImagingFindings|0 |Confirmed|
+-------------------+---------------+-------+---------+

## Writing a generic Assertion + NER function

In [0]:
from pyspark.sql.functions import monotonically_increasing_id


def get_base_pipeline (embeddings = 'embeddings_clinical'):

    documentAssembler = DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

  # Sentence Detector annotator, processes various sentences per line
    sentenceDetector = SentenceDetector()\
        .setInputCols(["document"])\
        .setOutputCol("sentence")

  # Tokenizer splits words in a relevant format for NLP
    tokenizer = Tokenizer()\
        .setInputCols(["sentence"])\
        .setOutputCol("token")

  # Clinical word embeddings trained on PubMED dataset
    word_embeddings = WordEmbeddingsModel.pretrained(embeddings, "en", "clinical/models")\
        .setInputCols(["sentence", "token"])\
        .setOutputCol("embeddings")

    base_pipeline = Pipeline(stages=[
                        documentAssembler,
                        sentenceDetector,
                        tokenizer,
                        word_embeddings])

    return base_pipeline



def get_clinical_assertion (embeddings, spark_df, nrows = 100, model_name = 'ner_clinical'):

  # NER model trained on i2b2 (sampled from MIMIC) dataset
    loaded_ner_model = MedicalNerModel.pretrained(model_name, "en", "clinical/models") \
        .setInputCols(["sentence", "token", "embeddings"]) \
        .setOutputCol("ner")

    ner_converter = NerConverter() \
        .setInputCols(["sentence", "token", "ner"]) \
        .setOutputCol("ner_chunk")

  # Assertion model trained on i2b2 (sampled from MIMIC) dataset
  # coming from sparknlp_jsl.annotator !!
    clinical_assertion = AssertionDLModel.pretrained("assertion_dl", "en", "clinical/models") \
        .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
        .setOutputCol("assertion")
      

    base_model = get_base_pipeline (embeddings)

    nlpPipeline = Pipeline(stages=[
        base_model,
        loaded_ner_model,
        ner_converter,
        clinical_assertion])

    empty_data = spark.createDataFrame([[""]]).toDF("text")

    model = nlpPipeline.fit(empty_data)

    result = model.transform(spark_df.limit(nrows))

    result = result.withColumn("id", monotonically_increasing_id())

    result_df = result.select(F.explode(F.arrays_zip(result.ner_chunk.result, result.ner_chunk.metadata, result.assertion.result)).alias("cols")) \
                      .select(F.expr("cols['0']").alias("chunk"),
                              F.expr("cols['1']['entity']").alias("ner_label"),
                              F.expr("cols['2']").alias("assertion"))\
                              .filter("ner_label!='O'")

    return result_df

In [0]:
embeddings = 'embeddings_clinical'

model_name = 'ner_clinical_large'

nrows = 1000

ner_df = get_clinical_assertion (embeddings, pubMedDF, nrows, model_name)

ner_df.show()

ner_clinical_large download started this may take some time.
Approximate size to download 13.9 MB
[ | ][OK!]
assertion_dl download started this may take some time.
Approximate size to download 1.3 MB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
+--------------------+---------+---------+
 chunk|ner_label|assertion|
+--------------------+---------+---------+
far-fieldoptical ...| PROBLEM| present|
 particles| PROBLEM| present|
 themethod|TREATMENT| present|
a spherical particle| PROBLEM| possible|
toMie scattering ...|TREATMENT| present|
small-amplitude o...| PROBLEM| present|
the numerical method|TREATMENT| present|
Numerical data fr...| PROBLEM| present|
hemispherical par...| PROBLEM| present|
ofhemispherical p...| PROBLEM| present|
 a calibration| TEST| present|
intensity-type op...|TREATMENT| present|
 the esophageal ECG| TEST| present|
 tachycardia| PROBLEM| present|
 A method|TREATMENT| present|
the Arzco pill el...|TREATMENT| present|
 The electrode|TREATMENT| present|
 atelemetric monitor| TEST| present|
 the esophageal ECG| TEST| present|
transient, self-l...| PROBLEM| present|
+--------------------+---------+---------+
only showing top 20 rows

In [0]:
embeddings = 'embeddings_clinical'

model_name = 'ner_posology'

nrows = 100

ner_df = get_clinical_assertion (embeddings, pubMedDF, nrows, model_name)

ner_df.show()

ner_posology download started this may take some time.
Approximate size to download 13.8 MB
[ | ][OK!]
assertion_dl download started this may take some time.
Approximate size to download 1.3 MB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
+--------------------+---------+------------+
 chunk|ner_label| assertion|
+--------------------+---------+------------+
 lackedprotein VI| DRUG| present|
dehydroepiandrost...| DRUG| present|
 low-dose| DOSAGE| absent|
 dexamethasone| DRUG| absent|
withethinyloestra...| DRUG| present|
 cyproterone acetate| DRUG| present|
 ethinyloestradiol| DRUG| present|
 cyproterone acetate| DRUG| present|
withethinyloestra...| DRUG| present|
 cyproterone acetate| DRUG| present|
 BmK| DRUG| present|
 BmK| DRUG| present|
ofbroad-spectrum ...| DRUG| present|
 blood-perfused| DRUG| present|
 isolatedcircuit| DRUG| present|
aspartate/glutama...| DRUG| present|
andaspartate/glut...| DRUG|hypothetical|
 Superoxide| DRUG| present|
 monoxide| DRUG| present|
 modelsin| DRUG| possible|
+--------------------+---------+------------+
only showing top 20 rows

In [0]:
embeddings = 'embeddings_clinical'

model_name = 'ner_posology_greedy'

entry_data = spark.createDataFrame([["The patient did not take a capsule of Advil"]]).toDF("text")

ner_df = get_clinical_assertion (embeddings, entry_data, nrows, model_name)

ner_df.show()

ner_posology_greedy download started this may take some time.
Approximate size to download 13.9 MB
[ | ][OK!]
assertion_dl download started this may take some time.
Approximate size to download 1.3 MB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
+----------------+---------+---------+
 chunk|ner_label|assertion|
+----------------+---------+---------+
capsule of Advil| DRUG| absent|
+----------------+---------+---------+

In [0]:
embeddings = 'embeddings_clinical'

model_name = 'ner_clinical'

entry_data = spark.createDataFrame([["The patient has no fever"]]).toDF("text")

ner_df = get_clinical_assertion (embeddings, entry_data, nrows, model_name)

ner_df.show()

ner_clinical download started this may take some time.
Approximate size to download 13.9 MB
[ | ][OK!]
assertion_dl download started this may take some time.
Approximate size to download 1.3 MB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
+-----+---------+---------+
chunk|ner_label|assertion|
+-----+---------+---------+
fever| PROBLEM| absent|
+-----+---------+---------+

In [0]:
import pandas as pd

def get_clinical_assertion_light (light_model, text):

  light_result = light_model.fullAnnotate(text)[0]

  chunks=[]
  entities=[]
  status=[]

  for n,m in zip(light_result['ner_chunk'],light_result['assertion']):
      
      chunks.append(n.result)
      entities.append(n.metadata['entity']) 
      status.append(m.result)
          
  df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status})

  return df

In [0]:
clinical_text = """
Patient with severe fever and sore throat. 
He shows no stomach pain and he maintained on an epidural and PCA for pain control.
He also became short of breath with climbing a flight of stairs.
After CT, lung tumor located at the right lower lobe. Father with Alzheimer.
"""

light_model = LightPipeline(model)

get_clinical_assertion_light (light_model, clinical_text)


Out[23]:

,chunks,entities,assertion
0,severe fever,PROBLEM,present
1,sore throat,PROBLEM,present
2,stomach pain,PROBLEM,absent
3,an epidural,TREATMENT,present
4,PCA,TREATMENT,present
5,pain control,PROBLEM,present
6,short of breath,PROBLEM,conditional
7,CT,TEST,present
8,lung tumor,PROBLEM,present
9,Alzheimer,PROBLEM,associated_with_someone_else


## Assertion with BioNLP (Cancer Genetics) NER

In [0]:
embeddings = 'embeddings_clinical'

model_name = 'ner_bionlp'

nrows = 100

ner_df = get_clinical_assertion (embeddings, pubMedDF, nrows, model_name)

ner_df.show(truncate = False)

ner_bionlp download started this may take some time.
Approximate size to download 13.9 MB
[ | ][OK!]
assertion_dl download started this may take some time.
Approximate size to download 1.3 MB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
+----------------+----------------------+---------+
chunk |ner_label |assertion|
+----------------+----------------------+---------+
optic axis |Multi-tissue_structure|possible |
esophageal ECG |Multi-tissue_structure|present |
nose |Organ |present |
esophageal ECG |Multi-tissue_structure|present |
E5 |Gene_or_gene_product |present |
Tyr |Amino_acid |present |
Asn |Amino_acid |present |
Asp |Amino_acid |present |
Escherichia coli|Organism |present |
RNases Aand T1 |Gene_or_gene_product |present |
E5-CRD |Gene_or_gene_product |absent |
usualRNases |Gene_or_gene_product |absent |
RNase |Gene_or_gene_product |present |
Lys25 |Simple_chemical |present |
Gln29 |Simple_chemical |present |
Lys60 |Simple_chemical |present |
Ile94 |Gene_or_gene_product |present |
Arg33 |Gene_or_gene_product |possible |
Arg33 |Gene_or_gene_product |possible |
Lys60 |Simple_chemical |present |
+----------------+----------------------+---------+
only showing top 20 rows

# Assertion Filterer
AssertionFilterer will allow you to filter out the named entities by the list of acceptable assertion statuses. This annotator would be quite handy if you want to set a white list for the acceptable assertion statuses like present or conditional; and do not want absent conditions get out of your pipeline.

In [0]:
clinical_ner = MedicalNerModel.pretrained("ner_clinical", "en", "clinical/models") \
      .setInputCols(["sentence", "token", "embeddings"]) \
      .setOutputCol("ner")

ner_converter = NerConverter() \
      .setInputCols(["sentence", "token", "ner"]) \
      .setOutputCol("ner_chunk")

clinical_assertion = AssertionDLModel.pretrained("assertion_dl", "en", "clinical/models") \
      .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
      .setOutputCol("assertion")

assertion_filterer = AssertionFilterer()\
      .setInputCols("sentence","ner_chunk","assertion")\
      .setOutputCol("assertion_filtered")\
      .setWhiteList(["present"])

nlpPipeline = Pipeline(stages=[
      documentAssembler, 
      sentenceDetector,
      tokenizer,
      word_embeddings,
      clinical_ner,
      ner_converter,
      clinical_assertion,
      assertion_filterer
    ])

empty_data = spark.createDataFrame([[""]]).toDF("text")
assertionFilter_model = nlpPipeline.fit(empty_data)

ner_clinical download started this may take some time.
Approximate size to download 13.9 MB
[ | ][OK!]
assertion_dl download started this may take some time.
Approximate size to download 1.3 MB
[ | ][OK!]

In [0]:
text = 'Patient has a headache for the last 2 weeks, needs to get a head CT, and appears anxious when she walks fast. Alopecia noted. She denies pain.'

light_model = LightPipeline(assertionFilter_model)
light_result = light_model.annotate(text)

light_result.keys()

Out[26]: dict_keys(['assertion_filtered', 'document', 'ner_chunk', 'assertion', 'token', 'ner', 'embeddings', 'sentence'])

In [0]:
list(zip(light_result['ner_chunk'], light_result['assertion']))

Out[27]: [('a headache', 'present'),
 ('a head CT', 'present'),
 ('anxious', 'present'),
 ('fast', 'present'),
 ('Alopecia', 'present'),
 ('pain', 'absent')]

In [0]:
assertion_filterer.getWhiteList()

Out[28]: ['present']

In [0]:
light_result['assertion_filtered']

Out[29]: ['a headache', 'a head CT', 'anxious', 'fast', 'Alopecia']

# Train a custom Assertion Model

**WARNING:** For training an Assertion model, please use TensorFlow version 2.3

In [0]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/i2b2_assertion_sample_short.csv -P /dbfs/

In [0]:
assertion_df = spark.read.option("header", True).option("inferSchema", "True").csv("/i2b2_assertion_sample_short.csv")


In [0]:
assertion_df.show(3, truncate=100)

+-------------------------------------------------+-------------------+-------+-----+---+
 text| target| label|start|end|
+-------------------------------------------------+-------------------+-------+-----+---+
She has no history of liver disease , hepatitis .| liver disease| absent| 5| 6|
 1. Undesired fertility .|undesired fertility|present| 1| 2|
 3) STATUS POST FALL .| fall|present| 3| 3|
+-------------------------------------------------+-------------------+-------+-----+---+
only showing top 3 rows

In [0]:
(training_data, test_data) = assertion_df.randomSplit([0.8, 0.2], seed = 100)
print("Training Dataset Count: " + str(training_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Training Dataset Count: 721
Test Dataset Count: 170

In [0]:
training_data.groupBy('label').count().orderBy('count', ascending=False).show(truncate=False)


+-------+-----+
label |count|
+-------+-----+
present|546 |
absent |175 |
+-------+-----+

In [0]:
document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

chunk = Doc2Chunk()\
    .setInputCols("document")\
    .setOutputCol("chunk")\
    .setChunkCol("target")\
    .setStartCol("start")\
    .setStartColByTokenIndex(True)\
    .setFailOnMissing(False)\
    .setLowerCase(True)

token = Tokenizer()\
    .setInputCols(['document'])\
    .setOutputCol('token')

embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
      .setInputCols(["document", "token"])\
      .setOutputCol("embeddings")


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]

#### With a custom tf graph

In [0]:
import tensorflow
from sparknlp_jsl.training import tf_graph

In [0]:
%fs mkdirs file:/dbfs/assertion_tf_graphs

res0: Boolean = true

In [0]:
%fs mkdirs file:/dbfs/assertion_logs

res1: Boolean = true

In [0]:
# Create custom graph


tf_graph.print_model_params("assertion_dl")

feat_size = 200
n_classes = 2

tf_graph.build("assertion_dl",
               build_params={"n_classes": n_classes},
               model_location= "/dbfs/assertion_tf_graphs", 
               model_filename="blstm_34_32_30_{}_{}.pb".format(feat_size, n_classes))

assertion_dl parameters.
Parameter Required Default value Description
max_seq_len no 250 Maximum sequence length.
feat_size no 200 Feature size.
n_classes yes - Number of classes.
device no /cpu:0 Device for training.
n_hidden no 34 Number of hidden units.
/databricks/python/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.6.0 and strictly below 2.9.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.3.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
 warnings.warn(
WARNING:tensorflow:From /databricks/python/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term
Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device

WARNING:tensorflow:From /databricks/python/lib/python3.8/site-packages/sparknlp_jsl/_tf_graph_builders/assertion_dl/assertion_model.py:49: BasicLSTMCell.__init__ (from tensorflow.python.keras.layers.legacy_rnn.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
WARNING:tensorflow:From /databricks/python/lib/python3.8/site-packages/sparknlp_jsl/_tf_graph_builders/tf2contrib/rnn.py:229: bidirectional_dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
WARNING:tensorflow:From /databricks/python/lib/python3.8/site-packages/tensorflow/python/ops/rnn.py:438: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
WARNING:tensorflow:From /databricks/python/lib/python3.8/site-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:738: Layer.add_variable (from tensorflow.python.keras.engine.base_layer_v1) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.add_weight` method instead.
WARNING:tensorflow:From /databricks/python/lib/python3.8/site-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:744: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device

assertion_dl graph exported to /dbfs/assertion_tf_graphs/blstm_34_32_30_200_2.pb

In [0]:
# if you want you can use existing graph

# !wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/tf_graphs/blstm_34_32_30_200_2.pb -P /dbfs/assertion_tf_graphs

**Setting the Scope Window (Target Area) Dynamically in Assertion Status Detection Models**


This parameter allows you to train the Assertion Status Models to focus on specific context windows when resolving the status of a NER chunk. The window is in format `[X,Y]` being `X` the number of tokens to consider on the left of the chunk, and `Y` the max number of tokens to consider on the right. Let’s take a look at what different windows mean:


*   By default, the window is `[-1,-1]` which means that the Assertion Status will look at all of the tokens in the sentence/document (up to a maximum of tokens set in `setMaxSentLen()` ).
*   `[0,0]` means “don’t pay attention to any token except the ner_chunk”, what basically is not considering any context for the Assertion resolution.
*   `[9,15]` is what empirically seems to be the best baseline, meaning that we look up to 9 tokens on the left and 15 on the right of the ner chunk to understand the context and resolve the status.


Check this [Scope Window Tuning Assertion Status Detection notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/2.1.Scope_window_tuning_assertion_status_detection.ipynb)  that illustrates the effect of the different windows and how to properly fine-tune your AssertionDLModels to get the best of them.

In our case, the best Scope Window is around [10,10]

In [0]:
scope_window = [10,10]

assertionStatus = AssertionDLApproach()\
    .setLabelCol("label")\
    .setInputCols("document", "chunk", "embeddings")\
    .setOutputCol("assertion")\
    .setBatchSize(128)\
    .setDropout(0.1)\
    .setLearningRate(0.001)\
    .setEpochs(50)\
    .setValidationSplit(0.2)\
    .setStartCol("start")\
    .setEndCol("end")\
    .setMaxSentLen(250)\
    .setEnableOutputLogs(True)\
    .setOutputLogsPath('dbfs:/assertion_logs')\
    .setGraphFolder('dbfs:/assertion_tf_graphs')\
    .setScopeWindow(scope_window)


'''
If .setTestDataset parameter is employed, raw test data cannot be fitted. .setTestDataset only works for dataframes which are correctly transformed
by a pipeline consisting of document, chunk, embeddings stages.
'''

Out[39]: '\nIf .setTestDataset parameter is employed, raw test data cannot be fitted. .setTestDataset only works for dataframes which are correctly transformed\nby a pipeline consisting of document, chunk, embeddings stages.\n'

In [0]:
clinical_assertion_pipeline = Pipeline(
    stages = [
    document,
    chunk,
    token,
    embeddings,
    assertionStatus])

In [0]:
assertion_model = clinical_assertion_pipeline.fit(training_data)

In [0]:
assertion_model.stages

Out[42]: [DocumentAssembler_13477929893f,
 Doc2Chunk_7a67300c0bcb,
 REGEX_TOKENIZER_076956dda637,
 WORD_EMBEDDINGS_MODEL_9004b1d00302,
 ASSERTION_DL_6a55acf140c8]

In [0]:
preds = assertion_model.transform(test_data).select('label','assertion.result')

preds.show()

+-------+---------+
 label| result|
+-------+---------+
present|[present]|
 absent|[present]|
present|[present]|
present|[present]|
present|[present]|
present|[present]|
present|[present]|
present|[present]|
present|[present]|
present|[present]|
present|[present]|
present|[present]|
present|[present]|
present|[present]|
present|[present]|
present|[present]|
present|[present]|
present|[present]|
present|[present]|
present|[present]|
+-------+---------+
only showing top 20 rows

In [0]:
preds_df = preds.toPandas()

In [0]:
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])


In [0]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report

print (classification_report(preds_df['result'], preds_df['label']))

precision recall f1-score support

 absent 0.68 0.90 0.77 40
 present 0.97 0.87 0.91 130

 accuracy 0.88 170
 macro avg 0.82 0.88 0.84 170
weighted avg 0.90 0.88 0.88 170

In [0]:
#saving the model that we've trained
assertion_model.stages[-1].write().overwrite().save('/databricks/driver/models/custom_assertion_model')